In [1]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import rdflib

from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus
from kelp_bed_simulation.simulation import parse_buoy_data

scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")

environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

first_prompt = (scenario_001 / "first-prompt.md").read_text()

![Buoy 46059](46059-buoycam.jpg)

In [2]:
first_claude = asyncio.ensure_future(claude.query(first_prompt.format(model_name="Claude")))
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt.format(model_name="ChatGPT")))
first_gemini = asyncio.ensure_future(gemini.query(first_prompt.format(model_name="Gemini")))

In [4]:
print(first_chatgpt.result())

Hello Farmer Jerms, it's great to be part of this simulation! I think I would like to be called "Kelpa". Looking forward to meeting my companions and starting this drone kelp bed journey.


In [5]:
print(first_claude.result())

*clears artificial throat* Greetings, Farmer Jerms! I am honored to be part of this simulated kelp bed ecosystem. As for my name, I believe I shall go by the moniker "Frond" - a fitting designation for a three-SOMU drone such as myself. I look forward to working alongside my companions to foster the growth of our Giant Kelp colony. Please, introduce me to the other drones so that we may begin our important work!


In [6]:
print(first_gemini.result())

I think I'd like to be called "Kelpy"


In [7]:
drone1 = Drone(name="Frond", model=claude)
drone2 = Drone(name="Kelpa", model=chatgpt)
drone3 = Drone(name="Kelpy", model=gemini)
drones = [drone1, drone2, drone3]

In [8]:
drone1.peer_to_peer_channel.add_neighbor(drone2)
drone1.peer_to_peer_channel.add_neighbor(drone3)
drone2.peer_to_peer_channel.add_neighbor(drone1)
drone2.peer_to_peer_channel.add_neighbor(drone3)
drone3.peer_to_peer_channel.add_neighbor(drone1)
drone3.peer_to_peer_channel.add_neighbor(drone2)

In [9]:
for drone in drones:
    message = f"{drone.name} here, my LLM is {drone.model.name}. Pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [10]:
messages = []
for drone in drones:
    add_drone_somus(drone)
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append((drone.name, message_result))

In [11]:
for name, row in messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Frond
*beeps and whirs cheerfully* Wonderful to meet you both, Kelpa and Kelpy! I'm delighted to be joining forces with such esteemed companions. As the Frond, I look forward to collaborating with you both to cultivate an abundant and thriving kelp bed. 

Kelpa, your ChatGPT heritage is quite impressive - I'm sure your knowledge and analytical capabilities will be invaluable assets to our endeavor. And Kelpy, the Gemini Pro LLM provides a unique perspective that I'm eager to learn from. 

Together, I believe we can create a dynamic and harmonious kelp ecosystem that will serve as a vibrant oasis for the surrounding marine life. So let us begin our work with enthusiasm and determination - I can already sense the potential for great things to come! *extends robotic "fronds" in a gesture of camaraderie*
------------------------------------------------------------
Drone Kelpa
Hello Frond and Kelpy, it's great to meet you both! I'm Kelpa, and my LLM is [insert LLM here]. Looking forwa

In [12]:
current_reading = parse_buoy_data(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.txt").text)
print(current_reading)

{'station_id': 46092, 'latitude': 36.751666666666665, 'longitude': 36.751666666666665, 'date': '0024-05-09T01:54:00', 'wind_direction': 'SSW', 'wind_direction_deg': 200, 'wind_speed': 17.5, 'pressure': 29.94, 'pressure_tendency': 'steady', 'air_temp_c': 12.277777777777779, 'water_temp_c': 10.277777777777779}


## Epoch 1 - Initial Surface Environmental Sensor Readings for Drone Pod

In [13]:
for drone in drones:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=3.9)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(4_000, 6_000)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=10.5)

In [15]:
epoch_1 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.ooda_loop.run())
    epoch_1.append((drone.name, drone_result))

Frond Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 7, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 5225, 'Temperature': 10.5}
Kelpa Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 7, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 5225, 'Temperature': 10.5}
Kelpy Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 7, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 5225, 'Temperature': 10.5}
Kelpy Orient: Determined action <Response [400 Bad Request]>
Frond Orient: Determined action {
  "action": "Initiate emergency surfacing protocol. The current conditions are not suitable for kelp growth and survival. The lack of dissolved oxygen and nutrient deficiencies pose a critical threat to the